Cargas librerías

In [16]:
! pip install mlxtend

In [17]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
from mlxtend.preprocessing import TransactionEncoder 
import re

In [18]:
#### conectar_base_de_Datos

conn=sql.connect('db_movies')
cur=conn.cursor()

In [19]:
#### ver tablas disponibles en base de datos ###

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

Sistema de recomendación basado en contenido un solo producto - Manual

In [42]:
#Se crea tabla master
movies=pd.read_sql('select * from movies_final', conn )

In [43]:
#Se realiza el .ifno para observar detalles de la tabla
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2441 entries, 0 to 2440
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  2441 non-null   int64 
 1   title    2441 non-null   object
 2   genres   2441 non-null   object
dtypes: int64(1), object(2)
memory usage: 57.3+ KB


In [44]:
# se convierte la columna genres en variables dummies
genres=movies['genres'].str.split('|')

te = TransactionEncoder()

genres = te.fit_transform(genres)

genres = pd.DataFrame(genres, columns = te.columns_)

movies = pd.concat([movies, genres], axis=1).drop(columns=["genres"])

movies.iloc[:,2:]=movies.iloc[:,2:].astype(int)

In [45]:

#Separar año del titulo
regex = r"\([0-9]+\)"
movies["year"]=movies.title.apply(lambda x:re.search(regex, x).group(0).replace("(","").replace(")",""))
movies["title"]=movies.title.apply(lambda x:re.sub(regex, "",x))

In [46]:
movies['year']=movies.year.astype('int')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2441 entries, 0 to 2440
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      2441 non-null   int64 
 1   title        2441 non-null   object
 2   Action       2441 non-null   int32 
 3   Adventure    2441 non-null   int32 
 4   Animation    2441 non-null   int32 
 5   Children     2441 non-null   int32 
 6   Comedy       2441 non-null   int32 
 7   Crime        2441 non-null   int32 
 8   Documentary  2441 non-null   int32 
 9   Drama        2441 non-null   int32 
 10  Fantasy      2441 non-null   int32 
 11  Film-Noir    2441 non-null   int32 
 12  Horror       2441 non-null   int32 
 13  IMAX         2441 non-null   int32 
 14  Musical      2441 non-null   int32 
 15  Mystery      2441 non-null   int32 
 16  Romance      2441 non-null   int32 
 17  Sci-Fi       2441 non-null   int32 
 18  Thriller     2441 non-null   int32 
 19  War          2441 non-null 

In [48]:
#Escalonar year
movies['year']=movies.year.astype('int')
sc=MinMaxScaler()
movies[["year"]]=sc.fit_transform(movies[['year']])

In [62]:
#se agrga la columna titulo a la tabla genres y se escalona
genres1= pd.get_dummies(movies,columns =['title'])
genres1 = genres1.drop(columns=['movieId','year'])

In [66]:
def recomendacion(movie = list(movies['title'])):
     
    ind_movie=movies[movies['title']==movie].index.values.astype(int)[0]   
    similar_movies = genres1.corrwith(genres1.iloc[ind_movie,:],axis=1)
    similar_movies = similar_movies.sort_values(ascending=False)
    top_similar_movies=similar_movies.to_frame(name="correlación").iloc[0:11,]
    top_similar_movies['title']=movies["title"]
    
    return top_similar_movies


print(interact(recomendacion))

interactive(children=(Dropdown(description='movie', options=('Toy Story ', 'Jumanji ', 'Grumpier Old Men ', 'F…

<function recomendacion at 0x0000017D9D012DD0>


Sistema de recomendación basado en contenido KNN un solo producto visto

In [70]:
#Entrenar modelo
model = neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
model.fit(genres)
dist, idlist = model.kneighbors(genres)

distancias=pd.DataFrame(dist)
id_list=pd.DataFrame(idlist)


In [74]:

def BookRecommender(movie_name = list(movies['title'].value_counts().index)):
    movie_list_name = []
    movie_id = movies[movies['title'] == movie_name].index
    movie_id = movie_id[0]
    for newid in idlist[movie_id]:
        movie_list_name.append(movies.loc[newid].title)
    return movie_list_name



print(interact(BookRecommender))

interactive(children=(Dropdown(description='movie_name', options=('King Kong ', 'Cape Fear ', 'Parent Trap, Th…

<function BookRecommender at 0x0000017D9E24F910>


In [80]:
usuarios=pd.read_sql('select distinct (userId) as userId from ratings_final',conn)

In [82]:
usuarios

,userId
0,1
1,3
2,4
3,5
4,6
...,...
460,606
461,607
462,608
463,609


In [ ]:
full=pd.read_sql('select * from movies_final', conn )

In [ ]:
def recomendar(user_id=list(usuarios['userId'].value_counts().index)):
    
    ratings=pd.read_sql('select *from ratings_final where user_id=:user',conn, params={'user':user_id})
    l_books_r=ratings['isbn'].to_numpy()
    genres1[['rating','title']]=[['isbn','book_title']]
    books_r=books_dum2[books_dum2['isbn'].isin(l_books_r)]
    books_r=books_r.drop(columns=['isbn','book_title'])
    books_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    centroide=books_r.groupby("indice").mean()
    
    
    books_nr=books_dum2[~books_dum2['isbn'].isin(l_books_r)]
    books_nr=books_nr.drop(columns=['isbn','book_title'])
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(books_nr)
    dist, idlist = model.kneighbors(centroide)
    
    ids=idlist[0]
    recomend_b=books.loc[ids][['book_title','isbn']]
    leidos=books[books['isbn'].isin(l_books_r)][['book_title','isbn']]
    
    return recomend_b

recomendar(52853)


print(interact(recomendar))